In [1]:
from keras.datasets import cifar10

Using TensorFlow backend.


In [2]:
import tensorflow as tf
import tensorflow.contrib.eager as tfe

In [3]:
import numpy as np
import os
import sys
import time

In [4]:
tf.enable_eager_execution()

In [5]:
tf.set_random_seed(42)

In [6]:
(X_train, Y_train), (X_test, Y_test) = cifar10.load_data()

In [7]:
(X_train, Y_train), (X_test, Y_test) = (X_train[:20000], Y_train[:20000]), (X_test[:2000], Y_test[:2000])

In [8]:
from scipy.ndimage.interpolation import zoom

In [9]:
def scale_dataset(x):
    new_x = []
    for i in range(len(x)):
        new_x.append(zoom(x[i], (8.0,8.0,1.0)))
        if (i%100)==0:
            print(f'Done: {i}')
    return new_x

In [10]:
x_train = scale_dataset(X_train)

Done: 0
Done: 100
Done: 200
Done: 300
Done: 400
Done: 500
Done: 600
Done: 700
Done: 800
Done: 900
Done: 1000
Done: 1100
Done: 1200
Done: 1300
Done: 1400
Done: 1500
Done: 1600
Done: 1700
Done: 1800
Done: 1900
Done: 2000
Done: 2100
Done: 2200
Done: 2300
Done: 2400
Done: 2500
Done: 2600
Done: 2700
Done: 2800
Done: 2900
Done: 3000
Done: 3100
Done: 3200
Done: 3300
Done: 3400
Done: 3500
Done: 3600
Done: 3700
Done: 3800
Done: 3900
Done: 4000
Done: 4100
Done: 4200
Done: 4300
Done: 4400
Done: 4500
Done: 4600
Done: 4700
Done: 4800
Done: 4900
Done: 5000
Done: 5100
Done: 5200
Done: 5300
Done: 5400
Done: 5500
Done: 5600
Done: 5700
Done: 5800
Done: 5900
Done: 6000
Done: 6100
Done: 6200
Done: 6300
Done: 6400
Done: 6500
Done: 6600
Done: 6700
Done: 6800
Done: 6900
Done: 7000
Done: 7100
Done: 7200
Done: 7300
Done: 7400
Done: 7500
Done: 7600
Done: 7700
Done: 7800
Done: 7900
Done: 8000
Done: 8100
Done: 8200
Done: 8300
Done: 8400
Done: 8500
Done: 8600
Done: 8700
Done: 8800
Done: 8900
Done: 9000
Done: 9100


In [11]:
x_test = scale_dataset(X_test)

Done: 0
Done: 100
Done: 200
Done: 300
Done: 400
Done: 500
Done: 600
Done: 700
Done: 800
Done: 900
Done: 1000
Done: 1100
Done: 1200
Done: 1300
Done: 1400
Done: 1500
Done: 1600
Done: 1700
Done: 1800
Done: 1900


In [12]:
x_train = tf.convert_to_tensor(x_train, np.float32)
x_test = tf.convert_to_tensor(x_test, np.float32)
y_train = tf.convert_to_tensor(Y_train, np.float32)
y_test = tf.convert_to_tensor(Y_test, np.float32)

In [13]:
total = x_train.numpy().shape[0]
train_len = int(total*0.7)
val_len = total-train_len
train_len, val_len

(14000, 6000)

In [14]:
IMG_SHAPE = [256,256,3]
NUM_CLASSES = 10

# training hyperparameters
LEARNING_RATE = 1e-4
MOMENTUM = 0.9
RMSPROP_DECAY = 0.9     
RMSPROP_EPSILON = 1.0              
BATCH_SIZE = 128
EPOCHS = 5
DISPLAY_STEP = 5 #10
VALIDATION_STEP = 50 #1000
SAVE_STEP = 10 #100
CKPT_PATH = './ckpt_base'
CKPT_PREFIX = os.path.join(CKPT_PATH, 'ckpt')
SUMMARY_PATH = './summary_base'

# net architecture hyperparamaters
LAMBDA = 5e-4 #for weight decay
DROPOUT = 0.5

# test hyper parameters
K_PATCHES = 5
TOP_K = 5

In [15]:
dataset = tf.data.Dataset.from_tensor_slices((x_train[:train_len], y_train[:train_len]))
dataset = dataset.shuffle(100).batch(BATCH_SIZE)
data_it = dataset.make_one_shot_iterator()

In [16]:
testset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
testset = testset.shuffle(10).batch(1)
test_it = testset.make_one_shot_iterator()

In [17]:
valset = tf.data.Dataset.from_tensor_slices((x_train[train_len:], y_train[train_len:]))
valset = valset.shuffle(50).batch(BATCH_SIZE)
val_it = valset.make_one_shot_iterator()

In [18]:
class AlexNet(tfe.Network):

    def __init__(self, training):
        super(AlexNet, self).__init__()
        self.training = training

        # convolutional layers

        conv_init = tf.contrib.layers.xavier_initializer_conv2d()

        self.conv1 = self.track_layer(tf.layers.Conv2D(96, 11, 4, 'SAME', 
                                                        activation=tf.nn.relu, 
                                                        kernel_initializer=conv_init))
        self.pool1 = self.track_layer(tf.layers.MaxPooling2D(3, 2, 'VALID'))

        self.conv2 = self.track_layer(tf.layers.Conv2D(256, 5, 1, 'SAME', 
                                                        activation=tf.nn.relu,
                                                        kernel_initializer=conv_init))
        self.pool2 = self.track_layer(tf.layers.MaxPooling2D(3, 2, 'VALID'))

        self.conv3 = self.track_layer(tf.layers.Conv2D(384, 3, 1, 'SAME', 
                                                        activation=tf.nn.relu,
                                                        kernel_initializer=conv_init))

        self.conv4 = self.track_layer(tf.layers.Conv2D(384, 3, 1, 'SAME', 
                                                        activation=tf.nn.relu,
                                                        kernel_initializer=conv_init))

        self.conv5 = self.track_layer(tf.layers.Conv2D(256, 3, 1, 'SAME', 
                                                        activation=tf.nn.relu,
                                                        kernel_initializer=conv_init))
        self.pool5 = self.track_layer(tf.layers.MaxPooling2D(3, 2, 'VALID'))

        # fully connected layers

        
        fc_init = tf.contrib.layers.xavier_initializer()

        self.fc1 = self.track_layer(tf.layers.Dense(4096, 
                                                        activation=tf.nn.relu,
                                                        kernel_initializer=fc_init))
        self.drop1 = self.track_layer(tf.layers.Dropout(DROPOUT))
        
        self.fc2 = self.track_layer(tf.layers.Dense(4096, 
                                                        activation=tf.nn.relu,
                                                        kernel_initializer=fc_init))
        self.drop2 = self.track_layer(tf.layers.Dropout(DROPOUT))

        
        
        self.out = self.track_layer(tf.layers.Dense(NUM_CLASSES,
                                                        kernel_initializer=fc_init))


    def call(self, x):
        """ Function that executes the model """
        output = self.conv1(x)
#         print(f'Conv1: {output.numpy().shape}')
        output = tf.nn.lrn(output, depth_radius=2, bias=1.0, alpha=2e-05, beta=0.75)
        output = self.pool1(output)
#         print(f'Pool1: {output.numpy().shape}')

        output = self.conv2(output)
#         print(f'Conv2: {output.numpy().shape}')
        output = tf.nn.lrn(output, depth_radius=2, bias=1.0, alpha=2e-05, beta=0.75)
        output = self.pool2(output)
#         print(f'Pool2: {output.numpy().shape}')

        output = self.conv3(output)
#         print(f'Conv3: {output.numpy().shape}')

        output = self.conv4(output)
#         print(f'Conv4: {output.numpy().shape}')
        output_conv4 = output 

        output = self.conv5(output)
#         print(f'Conv5: {output.numpy().shape}')
        output_conv5 = output 
        output = self.pool5(output)
#         print(f'Pool5: {output.numpy().shape}')

        output = tf.layers.flatten(output)
#         print(f'Flatten1: {output.numpy().shape}')

        output = self.fc1(output)
#         print(f'FC1: {output.numpy().shape}')
        if self.training:
            output = self.drop1(output)
            
        output = self.fc2(output)
#         print(f'FC2: {output.numpy().shape}')
        if self.training:
            output = self.drop2(output)
            
        output = self.out(output)

        return output

In [19]:
model = AlexNet(True)

Instructions for updating:
Please inherit from `tf.keras.Model`, and see its documentation for details. `tf.keras.Model` should be a drop-in replacement for `tfe.Network` in most cases, but note that `track_layer` is no longer necessary or supported. Instead, `Layer` instances are tracked on attribute assignment (see the section of `tf.keras.Model`'s documentation on subclassing). Since the output of `track_layer` is often assigned to an attribute anyway, most code can be ported by simply removing the `track_layer` calls.

`tf.keras.Model` works with all TensorFlow `Layer` instances, including those from `tf.layers`, but switching to the `tf.keras.layers` versions along with the migration to `tf.keras.Model` is recommended, since it will preserve variable names. Feel free to import it with an alias to avoid excess typing :).

Please inherit from `tf.keras.Model`, and see its documentation for details. `tf.keras.Model` should be a drop-in replacement for `tfe.Network` in most cases, but

In [20]:
writer = tf.contrib.summary.create_summary_file_writer(SUMMARY_PATH)

In [21]:
optimizer = tf.train.MomentumOptimizer(learning_rate=LEARNING_RATE, momentum=MOMENTUM)
# optimizer = tf.train.RMSPropOptimizer(learning_rate=LEARNING_RATE, RMSPROP_DECAY,momentum=MOMENTUM,epsilon=RMSPROP_EPSILON)
# optimizer = tf.train.AdamOptimizer(learning_rate=LEARNING_RATE)
# opt = tf.train.GradientDescent(learning_rate=LEARNING_RATE)

In [22]:
root = tfe.Checkpoint(optimizer=optimizer, model=model, optimizer_step=tf.train.get_or_create_global_step())

In [23]:
def loss(model, mode, x, y):
    logits = model(x)
    y = list(y.numpy().reshape(y.numpy().shape[0],))
    y = tf.one_hot(y, NUM_CLASSES)
    loss_value = tf.losses.softmax_cross_entropy(onehot_labels=y, logits=logits)
#     loss_value = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = y), name="cross_entropy_loss")
    weight_decay = tf.reduce_sum(LAMBDA * tf.stack([tf.nn.l2_loss(v) for v in model.variables]))

    total_loss = loss_value + weight_decay

    tf.contrib.summary.scalar(mode, '/loss', total_loss)

    return total_loss

In [24]:
def accuracy(model, mode, x, y):
#     pred = tf.nn.softmax(model(x))
#     y = tf.one_hot(y, NUM_CLASSES)
#     accuracy_value = tf.reduce_sum(
#                 tf.cast(
#                     tf.equal(
#                         tf.argmax(pred, axis=1, output_type=tf.int64),
#                         tf.argmax(y, axis=1, output_type=tf.int64)
#                     ),
#                     dtype=tf.float32
#                 ) 
#             ) / float(pred.shape[0].value)

    pred = tf.cast(tf.argmax(tf.nn.softmax(model(x)),axis=1), dtype=tf.float32)
    equality = tf.equal(pred, y)
    accuracy_value = tf.reduce_mean(tf.cast(equality, tf.float32))
    
    tf.contrib.summary.scalar(mode, '/accuracy', accuracy_value)

    return accuracy_value

In [25]:
def format_time(time):
    """ It formats a datetime to print it
        Args:
            time: datetime
        Returns:
            a formatted string representing time
    """
    m, s = divmod(time, 60)
    h, m = divmod(m, 60)
    d, h = divmod(h, 24)
    return ('{:02d}d {:02d}h {:02d}m {:02d}s').format(int(d), int(h), int(m), int(s))

In [26]:
global_step = tf.train.get_global_step()

In [27]:
all_variables = (model.variables + optimizer.variables() + [global_step])

## training

In [ ]:
start_time = time.time()
step_time = 0.0
global_step = 0
# with writer.as_default():
#     with tf.contrib.summary.record_summaries_every_n_global_steps(DISPLAY_STEP):

for epoch in range(EPOCHS):
    for (batch_i,datum) in enumerate(data_it):
        global_step = tf.train.get_global_step()
#                 global_step = tf.train.get_or_create_global_step()
        step = global_step.numpy() + 1
#         global_step += 1
        print(f'step: {step}')
        step_start_time = int(round(time.time() * 1000))
        optimizer.minimize(lambda: loss(model, 'train', datum[0], datum[1]), global_step=global_step)

        step_end_time = int(round(time.time() * 1000))
        step_time = step_time + step_end_time - step_start_time

        if (step % DISPLAY_STEP) == 0:
            l = loss(model, 'train', datum[0], datum[1])
            a = accuracy(model, 'train', datum[0], datum[1]).numpy()
            print ('Epoch: {:03d} Step/Batch: {:03d} Step mean time: {:04d}ms \nLoss: {:.7f} Training accuracy: {:.4f}'.format(epoch, step, int(step_time / step), l, a))

        if (step % VALIDATION_STEP) == 0:
            val_images, val_labels = val_it.get_next()
            l = loss(model, 'val', val_images, val_labels)
            a = accuracy(model, 'val', val_images, val_labels).numpy()
            int_time = time.time() - start_time
            print ('Elapsed time: {} --- Loss: {:.7f} Validation accuracy: {:.4f}'.format(format_time(int_time), l, a))

        if (step % SAVE_STEP) == 0:
            tfe.Saver(all_variables).save(os.path.join(CKPT_PATH, 'net.ckpt'), global_step=global_step)
            print('Variables saved')

step: 1
step: 2
step: 3
step: 4
step: 5
Epoch: 000 Step/Batch: 005 Step mean time: 25230ms 
Loss: 11.6123896 Training accuracy: 0.1064
step: 6
step: 7
step: 8
step: 9
step: 10
Epoch: 000 Step/Batch: 010 Step mean time: 25581ms 
Loss: 5.7429338 Training accuracy: 0.0978
Variables saved
step: 11
step: 12
step: 13
step: 14
step: 15
Epoch: 000 Step/Batch: 015 Step mean time: 27602ms 
Loss: 5.1828890 Training accuracy: 0.0961
step: 16
step: 17
step: 18
step: 19
step: 20
Epoch: 000 Step/Batch: 020 Step mean time: 26802ms 
Loss: 5.1045966 Training accuracy: 0.1005
Variables saved
step: 21
step: 22
step: 23
step: 24
step: 25
Epoch: 000 Step/Batch: 025 Step mean time: 26074ms 
Loss: 5.0344782 Training accuracy: 0.1218
step: 26
step: 27
step: 28
step: 29
step: 30
Epoch: 000 Step/Batch: 030 Step mean time: 25606ms 
Loss: 5.0422287 Training accuracy: 0.0978
Variables saved
step: 31
step: 32
step: 33
step: 34
step: 35
Epoch: 000 Step/Batch: 035 Step mean time: 25116ms 
Loss: 5.0483422 Training accu

## testing

In [196]:
tfe.Saver(model.variables).restore(tf.train.latest_checkpoint(CKPT_PATH))

INFO:tensorflow:Restoring parameters from ./ckpt_base/net.ckpt-50


In [236]:
ins = test_it.get_next()
x = ins[0]
y = ins[1]

In [245]:
def test_accuracy(x, y):
    pred = tf.cast(tf.argmax(tf.nn.softmax(model(x)),axis=1), dtype=tf.float32)
    equality = tf.equal(pred, y)
    accuracy = tf.reduce_mean(tf.cast(equality, tf.float32))
    return accuracy

In [254]:
test_examples = x_test.numpy().shape[0]

total_acc = 0.

for (ex_i, datum) in enumerate(test_it):
#     print(ex_i)
    images = datum[0]
    label = datum[1]

    acc = test_accuracy(images, label)
    total_acc += acc

    if (ex_i % 100) == 0:
        print ('Examples done: {:5d}/{} ---- Acc: {:.4f} '.format(ex_i + 1, test_examples, total_acc / (ex_i + 1)))

print ('---- Final accuracy ----')
print ('Acc: {:.4f} '.format(total_acc / test_examples))
print ('Error rate: {:.4f} '.format(1 - (total_acc / test_examples) ))


Examples done:     1/1000 ---- Acc: 0.0000 
Examples done:   101/1000 ---- Acc: 0.3465 
Examples done:   201/1000 ---- Acc: 0.3134 
Examples done:   301/1000 ---- Acc: 0.2990 
Examples done:   401/1000 ---- Acc: 0.3092 
Examples done:   501/1000 ---- Acc: 0.3054 
Examples done:   601/1000 ---- Acc: 0.3278 
Examples done:   701/1000 ---- Acc: 0.3224 
Examples done:   801/1000 ---- Acc: 0.3159 
Examples done:   901/1000 ---- Acc: 0.3274 
---- Final accuracy ----
Acc: 0.3260 
Error rate: 0.6740 
